In [70]:
import pandas as pd
from ast import literal_eval
import itertools
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [71]:
app = pd.read_csv("Data/app_list.csv")
steam_data = pd.read_csv("Data/steam_app_data.csv")
steam_spy = pd.read_csv("Data/steamspy_data.csv")

In [72]:
steam_data.drop(['type', 'controller_support', 'screenshots', 'achievements', "support_info", "dlc", 'fullgame', 'legal_notice', 'drm_notice', 'ext_user_account_notice', 'demos','price_overview','packages','package_groups', 'header_image', 'website', 'metacritic', 'reviews', 'background', 'detailed_description', 'publishers', 'background', 'movies', 'recommendations', 'supported_languages', 'pc_requirements', 'mac_requirements', 'linux_requirements'], axis=1, inplace=True)


In [73]:
steam_data = steam_data[steam_data['is_free'].notna()]
steam_data = steam_data[steam_data['required_age'].notna()]

In [74]:
def process_tags(df, export=False):
    if export: 

        tag_data = df[['appid', 'tags']].copy()

        def parse_export_tags(x):
            x = literal_eval(x)

            if isinstance(x, dict):
                return x
            elif isinstance(x, list):
                return {}
            else:
                raise TypeError('Something other than dict or list found')

        tag_data['tags'] = tag_data['tags'].apply(parse_export_tags)

        cols = set(itertools.chain(*tag_data['tags']))

        for col in sorted(cols):
            col_name = col.lower().replace(' ', '_').replace('-', '_').replace("'", "")

            tag_data[col_name] = tag_data['tags'].apply(lambda x: x[col] if col in x.keys() else 0)

        tag_data = tag_data.drop('tags', axis=1)

    def parse_tags(x):
        x = literal_eval(x)

        return ';'.join(list(x.keys())[:3]) if isinstance(x, dict) else np.nan

    df['tags'] = df['tags'].apply(parse_tags)

    # rows with null tags seem to be superseded by newer release, so remove (e.g. dead island)
    df = df[df['tags'].notnull()]

    return df


def process(df):
    df = df.copy()
    
    # handle missing values
    df = df[(df['name'].notnull()) & (df['name'] != 'none')]
    df = df[df['developer'].notnull()]
    df = df[df['languages'].notnull()]
    df = df[df['price'].notnull()]
    
    # remove unwanted columns
    df = df.drop([
        'genre', 'developer', 'publisher', 'score_rank', 'userscore', 'average_2weeks',
        'median_2weeks', 'price', 'initialprice', 'discount', 'ccu'
    ], axis=1)
    
    # keep top tags, exporting full tag data to file
    df = process_tags(df, export=True)
    
    # reformat owners column
    df['owners'] = df['owners'].str.replace(',', '').str.replace(' .. ', '-')
    
    return df

steam_spy = process(steam_spy)

Exported tag data to '../data/exports/steamspy_tag_data.csv'


In [75]:
steam_data=steam_data.dropna().reset_index(drop=True)

In [76]:
steam_spy=steam_spy.dropna().reset_index(drop=True)

In [77]:
merged = steam_data.merge(steam_spy, left_on='steam_appid', right_on='appid', suffixes=('', '_steamspy'))

In [78]:
merged.drop(['appid','name_steamspy', 'categories', 'genres', 'content_descriptors'], axis=1, inplace=True)

In [79]:
merged.to_csv('clean_data.csv', index=False)